In [ ]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [ ]:
motion_data = mr.process_apple_watch_csv("datasets/kawano/20240517/MotionData_20240517_220252.csv")

In [ ]:
check = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_check")
circle = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_circle")
cross = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_cross")
tri = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_tri")

In [ ]:
eye_data = mr.process_tobii_csv("datasets/kawano/20240517/kawano Recording002.csv")

In [ ]:
times_x, times_y, times_z = []
epsilon = 0.
for train in circle:
    _, times = mr.plot_spring(motion_data['SG_AccelerationX'], train['SG_AccelerationX'], motion_data['Timestamp'], epsilon)
    times_x.append(times)
    _, times = mr.plot_spring(motion_data['SG_AccelerationY'], train['SG_AccelerationY'], motion_data['Timestamp'], epsilon)
    times_y.append(times)
    _, times = mr.plot_spring(motion_data['SG_AccelerationZ'], train['SG_AccelerationZ'], motion_data['Timestamp'], epsilon)
    times_z.append(times)

In [ ]:
S = 0.1
N = 50

In [ ]:
mask = motion_data['EuclideanNorm'] <= S
# 連続する True のブロックにグループ番号を割り当てる
mask_shifted = mask.ne(mask.shift(1))  # 状態が変わる箇所を検出
cumsum = mask_shifted.cumsum()  # 変化点で番号を増やしていく

In [ ]:
# N サンプル以上連続する True のグループを見つける
N = 50  # 例として 5 を連続カウントの閾値に設定
groups = mask.groupby(cumsum).transform('sum')  # グループごとの True の合計数を求める
to_remove = groups >= N  # N サンプル以上のグループを True とする

# 条件に一致する区間を削除
filtered_data = motion_data[~to_remove]

In [ ]:
filtered_data['Timestamp']

In [ ]:
pathes, times = mr.plot_spring(filtered_data['SG_AccelerationX'], circle1['AccelerationX'], filtered_data['Timestamp'], 0.1)